In [ ]:
import pandas
import numpy as np
import math
import os 

from matplotlib import pyplot as plt
import matplotlib
import csv 
import xlsxwriter
import scipy
from sklearn.metrics import r2_score
from dipy.io.image import load_nifti, save_nifti

In [ ]:
def unique(list1): 
    # initialize a null list
    unique_list = []
  
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)           
    return unique_list


def calculate_and_create_CI_mat(df):
    K1 = df['K1'].tolist()
    N1 = df['N1'].tolist()
    K2 = df['K2'].tolist()
    N2 = df['N2'].tolist() 
    K1 = np.array(K1)
    K2 = np.array(K2)
    N1 = np.array(N1)
    N2 = np.array(N2)
    
    N = np.add(N1,N2)
    p1 = np.divide(K1,N1)
    p2 = np.divide(K2,N2)
    
    p = (N1*p1 + N2*p2)/N
    
    var_p = (p*(1-p))/N
    
    z_score = 1.96   
    CI_L = p - z_score*np.sqrt(var_p)
    CI_H = p + z_score*np.sqrt(var_p)
       
    return CI_L, CI_H


def create_correlation_matrix(path):
    df = pandas.read_excel(path, engine='openpyxl')
    
    print("found: ", df.columns.ravel())
    names = df['SEED2TARGET'].tolist()
    cps_raw = df['CPab'].tolist()
    
    seeds = []
    targets = []
    vals = []

    for i in range(0,len(names)):
        test_str = names[i]
        substring = '_2022'
        first = test_str.split(substring)[0]
        substring2 = '.2.'
        second = test_str.split(substring2)[1]
        third = second.split(substring)[0]
        seeds.append(first)
        targets.append(third)
        vals.append(cps_raw[i]*100)    # probably not a percent but raw val
        
        
    unique_seeds = unique(seeds)
    unique_seeds.sort()
    cp_mat = np.zeros((len(unique_seeds),len(unique_seeds)), float)
    cp_stats_mat = np.zeros((len(unique_seeds),len(unique_seeds)), float)
    
    CI_L_mat = np.zeros((len(unique_seeds),len(unique_seeds)), float)
    CI_H_mat = np.zeros((len(unique_seeds),len(unique_seeds)), float)
    
    CI_L, CI_H = calculate_and_create_CI_mat(df) 
    print("dims of CI are: ", CI_L.shape)
    
    # to get CP_ab map 
    for i in range(0,len(seeds)):
        sd = seeds[i]
        targ = targets[i]
        val = vals[i]

        ind1 = unique_seeds.index(sd)
        ind2 = unique_seeds.index(targ)
        cp_mat[ind1,ind2] = val
        
        #### parse out confidence intervals of each element 
        ci_l_val = CI_L[i]
        ci_h_val = CI_H[i]
        CI_L_mat[ind1,ind2] = ci_l_val
        CI_H_mat[ind1,ind2] = ci_h_val
        
    for i in range(cp_mat.shape[0]):
        for j in range(cp_mat.shape[1]):           
            CI_L_ab = CI_L_mat[i,j]
            CI_H_ab = CI_H_mat[i,j]
              
            CI_L_a_bp = CI_L_mat[i,unique_seeds.index('BP_CENTER')]
            CI_H_a_bp = CI_H_mat[i,unique_seeds.index('BP_CENTER')]   
            CI_L_a_mcp = CI_L_mat[i,unique_seeds.index('RN_CENTER')] 
            CI_H_a_mcp = CI_H_mat[i,unique_seeds.index('RN_CENTER')] 
            CI_L_b_bp = CI_L_mat[j,unique_seeds.index('BP_CENTER')]
            CI_H_b_bp = CI_H_mat[j,unique_seeds.index('BP_CENTER')]
            CI_L_b_mcp = CI_L_mat[j,unique_seeds.index('RN_CENTER')]
            CI_H_b_mcp = CI_H_mat[j,unique_seeds.index('RN_CENTER')]
                      
            CI_trigger = True
            if max(CI_L_ab,CI_L_a_bp) <= min(CI_H_ab,CI_H_a_bp):
                CI_trigger = False
            if max(CI_L_ab,CI_L_a_mcp) <= min(CI_H_ab,CI_H_a_mcp):
                CI_trigger = False
            if max(CI_L_ab,CI_L_b_bp) <= min(CI_H_ab,CI_H_b_bp):
                CI_trigger = False
            if max(CI_L_ab,CI_L_b_mcp) <= min(CI_H_ab,CI_H_b_mcp):
                CI_trigger = False
                
            if CI_trigger == True:
                cp_stats_mat[i,j] = 1     # no overlap
            else:
                cp_stats_mat[i,j] = 0     # overlap!    
    return cp_mat, cp_stats_mat, unique_seeds


def normalize_and_convert_to_rgba(rgb_list, alpha=255):
    # Convert RGB and alpha values to numpy arrays
    rgb_array = np.array(rgb_list, dtype=float)
    alpha_array = np.full((len(rgb_list), 1), alpha, dtype=float)

    # Normalize to the range 0-1
    rgb_array /= 255.0
    alpha_array /= 255.0

    # Convert to RGBA
    rgba_array = np.hstack((rgb_array, alpha_array))

    return rgba_array.tolist()

In [ ]:
"""
GENERATE CI calculations and paper-ready table format for dAAN EXC cases

"""


n_digits = 6
cases = ["EXC007","EXC012"]
n_cases = len(cases)


row_list1 = ["mRt", "VTA", "PAG", "PTg", "DR", "PnO", "PBC", "MnR", "LC", "LDTg"]
col_list1 = ["IL","PaV", "Ret"]
row_list2 = ["mRt", "VTA", "PAG", "PTg", "DR", "PnO", "PBC", "MnR", "LC", "LDTg"]
col_list2 = ["TMN", "LHA", "SUM"]
row_list3 = ["mRt", "VTA", "PAG", "PTg", "DR", "PnO", "PBC", "MnR", "LC", "LDTg"]
col_list3 = ["NDB", "BNM_SI"]
row_list4 = ["mRt_L", "PTg_L", "PnO_L", "PBC_L", "LC_L", "LDTg_L"]
col_list4 = ["mRt_L", "PTg_L", "PnO_L", "PBC_L", "LC_L", "LDTg_L"]
row_list5 = ["mRt_R", "PTg_R", "PnO_R", "PBC_R", "LC_R", "LDTg_R"]
col_list5 = ["mRt_R", "PTg_R", "PnO_R", "PBC_R", "LC_R", "LDTg_R"]
row_list6 = ["mRt_L", "PTg_L", "PnO_L", "PBC_L", "LC_L", "LDTg_L", "mRt_R", "PTg_R", "PnO_R", "PBC_R", "LC_R", "LDTg_R"]
col_list6 = ["VTA", "PAG", "DR", "MnR"]
row_list7 = ["mRt_L", "PTg_L", "PnO_L", "PBC_L", "LC_L", "LDTg_L"]
col_list7 = ["mRt_R", "PTg_R", "PnO_R", "PBC_R", "LC_R", "LDTg_R"]
row_list8 = ["mRt", "VTA", "PAG", "PTg", "DR", "PnO", "PBC", "MnR", "LC", "LDTg"]
col_list8 = ["DMN_PMC", "DMN_MPFC", "DMN_IPL","DMN_LT","DMN_MT_L"]
row_list8 = ["mRt", "VTA", "PAG", "PTg", "DR", "PnO", "PBC", "MnR", "LC", "LDTg"]
col_list8 = ["DMN_PMC", "DMN_MPFC", "DMN_IPL","DMN_LT","DMN_MT_L"]
row_list9 = ["ser", "nor", "dop", "ace", "glut"]
col_list9 = ["DMN"]
row_list10 = ["VTA", "PAG", "DR", "MnR"]
col_list10 = ["VTA", "PAG", "DR", "MnR"]

row_list_list = (row_list1,row_list2,row_list3,row_list4,row_list5,row_list6,row_list7,row_list8,row_list10)
col_list_list = (col_list1,col_list2,col_list3,col_list4,col_list5,col_list6,col_list7,col_list8,col_list10)




### try xlsxwriter with bolded CI pass vals 
workbook = xlsxwriter.Workbook("/Users/markolchanyi/Desktop/table_test_05232023.xlsx")

book_format_true = workbook.add_format(properties={'bold': False, 'font_color': 'black'}))
book_format_false = workbook.add_format(properties={'bold': False, 'font_color': 'red'})

worksheet = workbook.add_worksheet()


row_counter = 0
for ind, row_list in enumerate(row_list_list):
    

    col_list = col_list_list[ind]
    
    out_array = np.zeros((len(row_list),len(col_list)*(n_cases+1)))
    std_array = np.zeros((len(row_list),len(col_list)*(n_cases+1)))
    mean_array = np.zeros((len(row_list),len(col_list)))
    stats_array = np.zeros((len(row_list),len(col_list)*(n_cases+1)))

    for j, col in enumerate(col_list):
        for ind, case in enumerate(cases): 

            path = "/Users/markolchanyi/Desktop/" + case + "_statistics.xlsx"

            mat, CI_mat, unique_seeds = create_correlation_matrix(path)

            for i, row in enumerate(row_list):
                out_array[i,(n_cases+1)*j + ind] = round(mat[unique_seeds.index(row),unique_seeds.index(col)],n_digits)
                if CI_mat[unique_seeds.index(row),unique_seeds.index(col)] == 1:   
                    stats_array[i,(n_cases+1)*j + ind] = 1

        for i in range(len(row_list)):
            avg = (out_array[i,(n_cases+1)*(j+1)-(n_cases+1)] + out_array[i,(n_cases+1)*(j+1)-n_cases] + out_array[i,(n_cases+1)*(j+1)-(n_cases-1)])/n_cases
            mean_array[i,j] = avg  
            std = np.std((out_array[i,(n_cases+1)*(j+1)-(n_cases+1)], out_array[i,(n_cases+1)*(j+1)-n_cases], out_array[i,(n_cases+1)*(j+1)-(n_cases-1)]))
            out_array[i,(n_cases+1)*(j+1)-1] = round(avg,n_digits)
            std_array[i,(n_cases+1)*(j+1)-1] = round(std,n_digits)

            ### make look clean, hacky 
            if stats_array[i,(n_cases+1)*(j+1)-(n_cases+1)] + stats_array[i,(n_cases+1)*(j+1)-n_cases] + stats_array[i,(n_cases+1)*(j+1)-(n_cases-1)] == n_cases:
                stats_array[i,(n_cases+1)*(j+1)-1] = 1
                          
    for i in range(out_array.shape[0]):
        worksheet.write(i+1+row_counter,0, str(row_list[i]) , book_format_true)
        for j in range(out_array.shape[1]):
            worksheet.write(row_counter,j+1, str(col_list[int(math.floor(j/(n_cases+1)))]), book_format_true)
            if stats_array[i,j] == 1:
                if (j + 1) % (len(cases)+1) == 0 and j > 1: 
                    std = std_array[i,j]
                    worksheet.write(i+1+row_counter, j+1, str(round(out_array[i,j],3)), book_format_true)
                else:
                    worksheet.write(i+1+row_counter, j+1, round(out_array[i,j],3), book_format_true)

            else: 
                if (j + 1) % (len(cases)+1) == 0 and j > 1: 
                    std = std_array[i,j]
                    worksheet.write(i+1+row_counter, j+1, str(round(out_array[i,j],3)), book_format_false)
                else:
                    worksheet.write(i+1+row_counter, j+1, round(out_array[i,j],3),book_format_false)
                
    row_counter += len(row_list) + n_cases
          
workbook.close() 

In [ ]:
#### load raw DMN files and extract individual structural and functional conn. ####
## for neurotransmitter-specific groups of AAN nodes ##

path_DMN = "./DMN_vol_file"
dmn_vol, dummy_affine = load_nifti(path_DMN, return_img=False)

AAN_roi_list = ["mRt", "VTA", "PAG", "PTg", "DR", "PnO", "PBC", "MnR", "LC", "LDTg"]
parsed_SC_list = ["ser", "nor", "dop", "ace", "glut"]
parsed_SC_vals = np.zeros_like(parsed_SC_list,dtype=np.float64)

atlas_label_path = "./AAN_label_MNI_path"

for roi in parsed_SC_list: 
    if roi == "ser": 
        vol1,dummy_affine = load_nifti(atlas_label_path + "MnR.nii.gz", return_img=False)
        vol2,dummy_affine = load_nifti(atlas_label_path + "DR.nii.gz", return_img=False)
        vol1[vol1 > 0.4] = 1
        vol1[vol2 > 0.4] = 1
        vol1[vol1 < 0.9] = 0
        FC_tmp = dmn_vol.copy()
        FC_tmp[vol1 == 0] = 0
        avg_fc_val = np.sum(FC_tmp)/np.count_nonzero(FC_tmp)
        parsed_SC_vals[parsed_SC_list.index(roi)] = avg_fc_val
    elif roi == "nor":
        vol1,dummy_affine = load_nifti(atlas_label_path + "LC.nii.gz", return_img=False)
        vol1[vol1>0.4] = 1
        vol1[vol1<0.9] = 0
        FC_tmp = dmn_vol.copy()
        FC_tmp[vol1 == 0] = 0
        avg_fc_val = np.sum(FC_tmp)/np.count_nonzero(FC_tmp)
        parsed_SC_vals[parsed_SC_list.index(roi)] = avg_fc_val
    elif roi == "dop": 
        vol1,dummy_affine = load_nifti(atlas_label_path + "VTA.nii.gz", return_img=False)
        vol1[vol1>0.4] = 1
        vol1[vol1<0.9] = 0
        FC_tmp = dmn_vol.copy()
        FC_tmp[vol1 == 0] = 0
        avg_fc_val = np.sum(FC_tmp)/np.count_nonzero(FC_tmp)
        parsed_SC_vals[parsed_SC_list.index(roi)] = avg_fc_val
    elif roi == "ace": 
        vol1,dummy_affine = load_nifti(atlas_label_path + "PTg.nii.gz", return_img=False)
        vol2,dummy_affine = load_nifti(atlas_label_path + "LDTg.nii.gz", return_img=False)
        vol1[vol1 > 0.4] = 1
        vol1[vol2 > 0.4] = 1
        vol1[vol1 < 0.9] = 0
        FC_tmp = dmn_vol.copy()
        FC_tmp[vol1 == 0] = 0
        avg_fc_val = np.sum(FC_tmp)/np.count_nonzero(FC_tmp)
        parsed_SC_vals[parsed_SC_list.index(roi)] = avg_fc_val
    elif roi == "glut": 
        vol1,dummy_affine = load_nifti(atlas_label_path + "mRt.nii.gz", return_img=False)
        vol2,dummy_affine = load_nifti(atlas_label_path + "PBC.nii.gz", return_img=False)
        vol3,dummy_affine = load_nifti(atlas_label_path + "PnO.nii.gz", return_img=False)
        vol1[vol1 > 0.4] = 1
        vol1[vol2 > 0.4] = 1
        vol1[vol3 > 0.4] = 1
        vol1[vol1 < 0.9] = 0
        FC_tmp = dmn_vol.copy()
        FC_tmp[vol1 == 0] = 0
        avg_fc_val = np.sum(FC_tmp)/np.count_nonzero(FC_tmp)
        parsed_SC_vals[parsed_SC_list.index(roi)] = avg_fc_val

In [ ]:
### FOR INDIVIDUAL AAN LABELS ###
AAN_roi_list = ["mRt", "VTA", "PAG", "PTg", "DR", "PnO", "PBC", "MnR", "LC", "LDTg"]
parsed_SC_list = AAN_roi_list
parsed_SC_vals = np.zeros_like(parsed_SC_list,dtype=np.float64)

for roi in parsed_SC_list: 
    vol1,dummy_affine = load_nifti(atlas_label_path + roi + ".nii.gz", return_img=False)
    vol1[vol1>0.4] = 1
    vol1[vol1<0.9] = 0
    FC_tmp = dmn_vol.copy()
    FC_tmp[vol1 == 0] = 0
    avg_fc_val = np.sum(FC_tmp)/np.count_nonzero(FC_tmp)
    parsed_SC_vals[parsed_SC_list.index(roi)] = avg_fc_val

In [ ]:
## PLOT NEUROTRANMITTER-SPECIFIC SF PLOTS
plt.rcParams["font.family"] = "Arial"
fig,ax = plt.subplots()

# Make a scatter plot

plt.plot(functional_mean_array, mean_array, 'o',c='black')
ax.tick_params(axis="y", direction='in', length=5)
ax.tick_params(axis="x", direction='in', length=5)
plt.xlabel("Functional Connectivity",fontweight="bold",fontsize=12)
plt.ylabel("Structural Connectivity Probability (%)",fontweight="bold",fontsize=12)

for i, txt in enumerate(row_list):
    if txt == "Glutamate":
        ax.annotate(txt, (functional_mean_array[i], mean_array[i]),xytext=(10, -55), textcoords='offset pixels')
    elif txt == "Serotonin":
        ax.annotate(txt, (functional_mean_array[i], mean_array[i]),xytext=(-250, -50), textcoords='offset pixels')
        
    else:
        ax.annotate(txt, (functional_mean_array[i], mean_array[i]),xytext=(0, 25), textcoords='offset pixels')
    
plt.savefig('./SF_neurotransmitter.png',dpi=400)
plt.show()

In [ ]:
## PLOT INIDIVIAL AAN NODE SPECIFIC SF ##
plt.rcParams["font.family"] = "Arial"
fig,ax = plt.subplots()
for i in range(len(functional_mean_array)):
    plt.scatter(functional_mean_array[i], mean_array[i], color=lut_array_rgba[i])
ax.tick_params(axis="y", direction='in', length=5)
ax.tick_params(axis="x", direction='in', length=5)
plt.xlabel("Functional Connectivity",fontweight="bold",fontsize=12)
plt.ylabel("Structural Connectivity Probability (%)",fontweight="bold",fontsize=12)

for i, txt in enumerate(row_list):
    if txt == "PBC":
        ax.annotate(txt, (functional_mean_array[i], mean_array[i]),xytext=(0, -65), textcoords='offset pixels')
    elif txt == "PAG":
        ax.annotate(txt, (functional_mean_array[i], mean_array[i]),xytext=(23, -10), textcoords='offset pixels')
    elif txt == "MnR":
        ax.annotate(txt, (functional_mean_array[i], mean_array[i]),xytext=(-125, 25), textcoords='offset pixels')
        
    else:
        ax.annotate(txt, (functional_mean_array[i], mean_array[i]),xytext=(0, 25), textcoords='offset pixels')
    
plt.savefig('./SF_individual_nuclei.png',dpi=400)
plt.show()